In [8]:
import pymongo
from pprint import pprint
import pandas as pd

# Create an instance of the Mongo Client
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Access the uk_food database and establishments collection
db = client.uk_food
establishments = db.establishments



In [9]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen"
business_type = establishments.find_one({"BusinessType": "Restaurant/Cafe/Canteen"}, {"BusinessTypeID": 1, "BusinessType": 1})
pprint(business_type)

# Extract the BusinessTypeID
business_type_id = business_type['BusinessTypeID']



{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('668f466b0bfeb5b4e26da006')}


In [10]:
# New restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": business_type_id,
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "",
    "AddressLine3": "",
    "AddressLine4": "Greenwich",
    "PostCode": "SE10 9HT",
    "FoodSafetyAuthorityID": "",
    "LocalAuthorityBusinessID": "",
    "LocalAuthorityCode": "",
    "LocalAuthorityEmailAddress": "",
    "LocalAuthorityName": "",
    "LocalAuthorityWebSite": "",
    "Longitude": -0.0125,
    "Latitude": 51.4816,
    "RatingDate": "",
    "RatingKey": "",
    "RatingValue": "",
    "Scores": {
        "ConfidenceInManagement": "",
        "Hygiene": "",
        "Structural": ""
    }
}

# Insert the new restaurant
insert_result = establishments.insert_one(new_restaurant)
print(f'Inserted new restaurant with id: {insert_result.inserted_id}')


Inserted new restaurant with id: 668f4dc717ec4bff2675ce2a


In [11]:
# Count documents in Dover
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f'Number of establishments in Dover: {dover_count}')


Number of establishments in Dover: 0


In [12]:
# Remove establishments in Dover
delete_result = establishments.delete_many({"LocalAuthorityName": "Dover"})
print(f'Number of documents deleted: {delete_result.deleted_count}')

# Verify removal
dover_count_after_removal = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f'Number of establishments in Dover after removal: {dover_count_after_removal}')


Number of documents deleted: 0
Number of establishments in Dover after removal: 0


In [13]:
# Convert Latitude and Longitude to decimal numbers
establishments.update_many(
    {}, 
    [
        {"$set": {"Longitude": {"$toDouble": "$Longitude"}, "Latitude": {"$toDouble": "$Latitude"}}}
    ]
)


UpdateResult({'n': 38790, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [16]:
# Coerce non-numeric RatingValue to null and convert to integer
establishments.update_many(
    {},
    [
        {
            "$set": {
                "RatingValue": {
                    "$cond": {
                        "if": {
                            "$regexMatch": {
                                "input": {"$toString": "$RatingValue"},
                                "regex": "^[0-9]+$"
                            }
                        },
                        "then": {"$toInt": "$RatingValue"},
                        "else": None
                    }
                }
            }
        }
    ]
)


UpdateResult({'n': 38790, 'nModified': 4, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [19]:
# Find establishments with a hygiene score of 20
hygiene_score_20 = establishments.find({"Scores.Hygiene": 20})

# Count documents
count_hygiene_score_20 = establishments.count_documents({"Scores.Hygiene": 20})
print(f'Number of establishments with a hygiene score of 20: {count_hygiene_score_20}')

# Display the first document using next()
first_document = hygiene_score_20.next()
pprint(first_document)

# Convert the cursor to a DataFrame
df_hygiene_score_20 = pd.DataFrame(list(hygiene_score_20))
print(f'Number of rows in the DataFrame: {len(df_hygiene_score_20)}')
df_hygiene_score_20.head(10)



Number of establishments with a hygiene score of 20: 0


StopIteration: 

In [20]:
# Find establishments in London with a RatingValue >= 4
london_high_rating = establishments.find({"LocalAuthorityName": {"$regex": "London"}, "RatingValue": {"$gte": 4}})

# Count documents
count_london_high_rating = establishments.count_documents({"LocalAuthorityName": {"$regex": "London"}, "RatingValue": {"$gte": 4}})
print(f'Number of establishments in London with a RatingValue >= 4: {count_london_high_rating}')

# Display the first document using next()
first_document = london_high_rating.next()
pprint(first_document)

# Convert the cursor to a DataFrame
df_london_high_rating = pd.DataFrame(list(london_high_rating))
print(f'Number of rows in the DataFrame: {len(df_london_high_rating)}')
df_london_high_rating.head(10)


Number of establishments in London with a RatingValue >= 4: 33
{'AddressLine1': 'Oak Apple Farm Building 103 Sheernes Docks',
 'AddressLine2': 'Sheppy Kent',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': "Charlie's",
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4627.439467780196,
 'FHRSID': 621707,
 'Latitude': None,
 'LocalAuthorityBusinessID': 'PI/000025307',
 'LocalAuthorityCode': '508',
 'LocalAuthorityEmailAddress': 'publicprotection@cityoflondon.gov.uk',
 'LocalAuthorityName': 'City of London Corporation',
 'LocalAuthorityWebSite': 'http://www.cityoflondon.gov.uk/Corporation/homepage.htm',
 'Longitude': None,
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'ME12',
 'RatingDate': '2021-10-18T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('668f466e0bfeb5b4e26dd553'),
 'geocode': {'latitude': '51.369321', 'longitude': '0.508551'}

,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,scores,SchemeType,geocode,RightToReply,Distance,NewRatingPending,meta,links,Longitude,Latitude
0,668f466e0bfeb5b4e26dd879,1130836,0,PI/000034075,Mv City Cruises Erasmus,Other catering premises,7841,Cherry Garden Pier,Cherry Garden Street Rotherhithe,Charlton,...,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': '0.249255999922752', 'latitude':...",,4627.873179,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...",None,None
1,668f466e0bfeb5b4e26de3c6,293783,0,PI/000002614,Benfleet Motor Yacht Club,Other catering premises,7841,Ferry Road,South Benfleet Essex,,...,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': '0.561954', 'latitude': '51.5438...",,4631.968418,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...",None,None
2,668f466f0bfeb5b4e26df1c6,1315095,0,PI/000036464,Coombs Catering t/a The Lock and Key,Restaurant/Cafe/Canteen,1,Leslie Ford House,Tilbury,Charlton,...,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': '0.346336990594864', 'latitude':...",,4636.846754,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...",None,None
3,668f466f0bfeb5b4e26df1c7,294474,0,PI/000014647,Tilbury Seafarers Centre,Restaurant/Cafe/Canteen,1,Tenants Row Tilbury Docks,Tilbury Essex,,...,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': '0.346325', 'latitude': '51.4640...",,4636.847650,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...",None,None
4,668f466f0bfeb5b4e26df8cf,294900,0,PI/000019066,Mv Valulla,Other catering premises,7841,Reeds River Cruises LtdKings ReachRiver Thames...,London,,...,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': '0.287555', 'latitude': '51.5040...",,4640.460834,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...",None,None
5,668f46700bfeb5b4e26e19dd,293756,0,PI/000002538,Tereza Joanne,Other catering premises,7841,Funcraft UK Ltd King George V Dock Woolwich Ma...,London,,...,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': '0.069286', 'latitude': '51.5011...",,4648.301822,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...",None,None
6,668f46700bfeb5b4e26e1da4,878523,0,PI/000029844,Brick Lane Brews,Restaurant/Cafe/Canteen,1,Air side London City Airport,London,,...,"{'Hygiene': 10, 'Structural': 5, 'ConfidenceIn...",FHRS,"{'longitude': '0.048006', 'latitude': '51.5037...",,4649.173485,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...",None,None
7,668f46700bfeb5b4e26e1dbe,293772,0,PI/000002585,The Nuance Group (UK) Limited,Retailers - other,4613,Duty Free Shop Passenger Terminal,London City AirportRoyal DocksLondon,,...,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': '0.047966', 'latitude': '51.5041...",,4649.188826,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...",None,None
8,668f46700bfeb5b4e26e1dbf,294606,0,PI/000016185,WH Smith,Retailers - other,4613,London City Airport,Silvertown London,,...,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': '0.0479655750095844', 'latitude'...",,4649.188850,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...",None,None
9,668f46700bfeb5b4e26e1dc0,295107,0,PI/000020749,City Bar & Grill,Restaurant/Cafe/Canteen,1,London City Airport (Airside),London,,...,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': '0.0479655750095844', 'latitude'..."

In [21]:
# Find Penang Flavours
penang_flavours = establishments.find_one({"BusinessName": "Penang Flavours"})
longitude = penang_flavours['Longitude']
latitude = penang_flavours['Latitude']

# Find top 5 establishments with RatingValue of 5, sorted by lowest hygiene score, nearest to Penang Flavours
top_5_near_penang = establishments.find({
    "RatingValue": 5,
    "Longitude": {"$gte": longitude - 0.01, "$lte": longitude + 0.01},
    "Latitude": {"$gte": latitude - 0.01, "$lte": latitude + 0.01}
}).sort("Scores.Hygiene", pymongo.ASCENDING).limit(5)

# Convert the cursor to a DataFrame
df_top_5_near_penang = pd.DataFrame(list(top_5_near_penang))
print(f'Number of rows in the DataFrame: {len(df_top_5_near_penang)}')
df_top_5_near_penang.head(10)


Number of rows in the DataFrame: 0


""


In [22]:
# Aggregate number of establishments with a hygiene score of 0 in each Local Authority
hygiene_score_0 = establishments.aggregate([
    {"$match": {"Scores.Hygiene": 0}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
])

# Convert the cursor to a DataFrame
df_hygiene_score_0 = pd.DataFrame(list(hygiene_score_0))
print(f'Number of rows in the DataFrame: {len(df_hygiene_score_0)}')
df_hygiene_score_0.head(10)


Number of rows in the DataFrame: 0


""
